In [39]:
#!pip install ipython-sql

In [1]:
%load_ext sql

In [2]:
%sql sqlite://

In [3]:
%%sql

CREATE TABLE sales (
  "customer_id" VARCHAR(1),
  "order_date" DATE,
  "product_id" INTEGER
);

INSERT INTO sales
  ("customer_id", "order_date", "product_id")
VALUES
  ('A', '2021-01-01', '1'),
  ('A', '2021-01-01', '2'),
  ('A', '2021-01-07', '2'),
  ('A', '2021-01-10', '3'),
  ('A', '2021-01-11', '3'),
  ('A', '2021-01-11', '3'),
  ('B', '2021-01-01', '2'),
  ('B', '2021-01-02', '2'),
  ('B', '2021-01-04', '1'),
  ('B', '2021-01-11', '1'),
  ('B', '2021-01-16', '3'),
  ('B', '2021-02-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-07', '3');
 

CREATE TABLE menu (
  "product_id" INTEGER,
  "product_name" VARCHAR(5),
  "price" INTEGER
);

INSERT INTO menu
  ("product_id", "product_name", "price")
VALUES
  ('1', 'sushi', '10'),
  ('2', 'curry', '15'),
  ('3', 'ramen', '12');
  

CREATE TABLE members (
  "customer_id" VARCHAR(1),
  "join_date" DATE
);

INSERT INTO members
  ("customer_id", "join_date")
VALUES
  ('A', '2021-01-07'),
  ('B', '2021-01-09');

 * sqlite://
Done.
15 rows affected.
Done.
3 rows affected.
Done.
2 rows affected.


[]

In [11]:
%%sql
SELECT * 
FROM sales;

 * sqlite://
Done.


customer_id,order_date,product_id
A,2021-01-01,1
A,2021-01-01,2
A,2021-01-07,2
A,2021-01-10,3
A,2021-01-11,3
A,2021-01-11,3
B,2021-01-01,2
B,2021-01-02,2
B,2021-01-04,1
B,2021-01-11,1


In [12]:
%%sql
SELECT * 
FROM menu;

 * sqlite://
Done.


product_id,product_name,price
1,sushi,10
2,curry,15
3,ramen,12


In [13]:
%%sql
SELECT * 
FROM members;

 * sqlite://
Done.


customer_id,join_date
A,2021-01-07
B,2021-01-09


# Question 1: What is the total amount each customer spent at the restaurant?

In [40]:
%%sql
SELECT s.customer_id,
    SUM(m.price) AS total_spent
FROM sales AS s
LEFT JOIN menu AS m
ON s.product_id = m.product_id
GROUP BY s.customer_id;

 * sqlite://
Done.


customer_id,total_spent
A,76
B,74
C,36


# Question 2: How many days has each customer visited the restaurant?

In [41]:
%%sql
SELECT customer_id,
    COUNT(DISTINCT order_date) AS num_days_visited
FROM sales
GROUP BY customer_id;

 * sqlite://
Done.


customer_id,num_days_visited
A,4
B,6
C,2


# Question 3: What was the first item from the menu purchased by each customer?

In [42]:
%%sql
WITH ranked_orders AS (
    SELECT DENSE_RANK() OVER(PARTITION BY s.customer_id ORDER BY s.order_date) AS rank,
        s.customer_id,
        s.order_date,
        m.product_name
    FROM sales AS s
    LEFT JOIN menu AS m
    ON s.product_id = m.product_id)

SELECT customer_id,
       product_name
FROM ranked_orders
WHERE rank = 1
GROUP BY customer_id, product_name;

 * sqlite://
Done.


customer_id,product_name
A,curry
A,sushi
B,curry
C,ramen


# Question 4: What is the most purchased item on the menu and how many times was it purchased by all customers?

In [44]:
%%sql
SELECT product_name,
    MAX(times_ordered) AS times_ordered
FROM (SELECT s.product_id,
      m.product_name,
      COUNT(*) AS times_ordered
      FROM sales AS s
      LEFT JOIN menu AS m
      ON s.product_id = m.product_id
      GROUP BY s.product_id);

 * sqlite://
Done.


product_name,times_ordered
ramen,8


# Question 5: Which item was the most popular for each customer?

In [45]:
%%sql
WITH ranked_items AS (
    SELECT s.customer_id, 
           m.product_name,
           COUNT(m.product_id) AS order_count,
           DENSE_RANK() OVER(PARTITION BY s.customer_id ORDER BY COUNT(s.customer_id) DESC) AS rank
    FROM menu AS m
    JOIN sales AS s
    ON m.product_id = s.product_id
    GROUP BY s.customer_id, m.product_name)

SELECT customer_id, product_name, order_count
FROM ranked_items 
WHERE rank = 1;

 * sqlite://
Done.


customer_id,product_name,order_count
A,ramen,3
B,curry,2
B,ramen,2
B,sushi,2
C,ramen,3


# Question 6: Which item was purchased first by the customer after they became a member?

In [46]:
%%sql
WITH member_sales AS (
    SELECT DENSE_RANK() OVER(PARTITION BY s.customer_id ORDER BY s.order_date) AS rank,
        s.customer_id,
        s.order_date,
        m.join_date,
        s.product_id
    FROM sales AS s
    LEFT JOIN members AS m
    ON s.customer_id = m.customer_id
    WHERE s.order_date>=m.join_date)

SELECT customer_id, 
       order_date,
       product_name AS first_member_order
    FROM member_sales
    LEFT JOIN menu
    ON member_sales.product_id=menu.product_id
    WHERE rank = 1;

 * sqlite://
Done.


customer_id,order_date,first_member_order
A,2021-01-07,curry
B,2021-01-11,sushi


# Question 7: Which item was purchased just before the customer became a member?

In [47]:
%%sql
WITH non_member_sales AS (
    SELECT DENSE_RANK() OVER(PARTITION BY s.customer_id ORDER BY order_date DESC) AS rank,
        s.customer_id,
        s.order_date,
        m.join_date,
        s.product_id
    FROM sales AS s
    LEFT JOIN members AS m
    ON s.customer_id = m.customer_id
    WHERE s.order_date < m.join_date)

SELECT customer_id, 
       order_date,
       product_name AS last_non_member_order
    FROM non_member_sales
    LEFT JOIN menu
    ON non_member_sales.product_id=menu.product_id
    WHERE rank = 1;

 * sqlite://
Done.


customer_id,order_date,last_non_member_order
A,2021-01-01,sushi
A,2021-01-01,curry
B,2021-01-04,sushi


# Question 8: What is the total items and amount spent for each member before they became a member?

In [13]:
%%sql
WITH non_member_sales AS (
    SELECT s.customer_id,
           s.order_date,
           m.join_date,
           s.product_id
    FROM sales AS s
    LEFT JOIN members AS m
    ON s.customer_id = m.customer_id
    WHERE s.order_date < m.join_date)

SELECT nms.customer_id,
       COUNT(nms.order_date) AS total_items,
       SUM(m2.price) AS amount_spent
FROM non_member_sales AS nms
LEFT JOIN menu AS m2
ON nms.product_id = m2.product_id
GROUP BY nms.customer_id;

 * sqlite://
Done.


customer_id,total_items,amount_spent
A,2,25
B,3,40


# Question 9: If each 1 dollar spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have? 

In [29]:
%%sql
WITH points AS (
    SELECT *,
           CASE WHEN product_id = 1 THEN price * 20
           ELSE price * 10 END AS total_points
    FROM menu)


SELECT s.customer_id,
       SUM(p.total_points) AS total_points
    FROM sales AS s
    LEFT JOIN points AS p
    ON s.product_id = p.product_id
    GROUP BY s.customer_id;

 * sqlite://
Done.


customer_id,total_points
A,860
B,940
C,360


# Question 10: In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

In [38]:
%%sql
WITH valid_dates AS (
    SELECT *, 
           DATE(join_date, '6 days') AS valid_date, 
           DATE(join_date,'start of month','+1 month','-1 day') AS last_date
    FROM members AS m)

SELECT s.customer_id,
       SUM(CASE WHEN s.order_date < v.join_date AND m.product_id = 1 THEN m.price * 20
       WHEN s.order_date BETWEEN v.join_date AND v.valid_date THEN m.price * 20
       ELSE m.price * 10 END) AS points
FROM sales AS s
JOIN valid_dates AS v
ON s.customer_id = v.customer_id
JOIN menu AS m
ON s.product_id = m.product_id
GROUP BY s.customer_id;

 * sqlite://
Done.


customer_id,points
A,1370
B,940
